In [4]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Color, Border, Side

def is_sheet_visible(file_path, sheet_name):
    try:
        workbook = load_workbook(file_path, read_only=True)
        sheet = workbook[sheet_name]
        return sheet.sheet_state == 'visible'
    except Exception as e:
        print(f"Error while checking sheet visibility: {e}")
        return False

def compare_excel_files(file1, file2):
    # Get visible sheet names from each ExcelFile
    with pd.ExcelFile(file1) as xls1, pd.ExcelFile(file2) as xls2:
        visible_sheets1 = [sheet for sheet in xls1.sheet_names if is_sheet_visible(file1, sheet)]
        visible_sheets2 = [sheet for sheet in xls2.sheet_names if is_sheet_visible(file2, sheet)]

        # Define pattern fill with thin vertical stripes (using 1/24 as width per stripe)
        pattern_fill = PatternFill(start_color="FFA07A", end_color="FFA07A", fill_type="darkVertical", patternType="darkVertical", fgColor=Color(rgb="FFA07A"))
        border_style = Border(left=Side(style='thick'), right=Side(style='thick'), top=Side(style='thick'), bottom=Side(style='thick'))  # Thick border

        # Compare each visible sheet
        for sheet in visible_sheets1:
            if sheet in visible_sheets2:
                try:
                    df1_sheet = pd.read_excel(file1, sheet_name=sheet, engine='openpyxl')
                    df2_sheet = pd.read_excel(file2, sheet_name=sheet, engine='openpyxl')
                except Exception as e:
                    print(f"Error reading sheets '{sheet}': {e}")
                    continue

                # Try to load workbook for file2 for modification
                try:
                    workbook = load_workbook(file2)
                except Exception as e:
                    print(f"Error loading workbook '{file2}' for modification: {e}")
                    return

                # Access the specific sheet in the workbook
                ws = workbook[sheet]

                # Compare row by row
                comparison_values = df1_sheet.values == df2_sheet.values
                rows, cols = np.where(comparison_values == False)

                for row, col in zip(rows, cols):
                    value_file1 = df1_sheet.iat[row, col]
                    value_file2 = df2_sheet.iat[row, col]
                    if pd.isna(value_file1) and pd.isna(value_file2):
                        continue  # Skip if both are NaN
                    else:
                        # Highlight the cell with differences using the pattern fill and border style
                        cell = ws.cell(row=row+2, column=col+1)  # Adjust for 1-based index in openpyxl
                        cell.fill = pattern_fill
                        cell.border = border_style

                # Save the modified workbook
                try:
                    workbook.save(file2)
                except Exception as e:
                    print(f"Error saving workbook '{file2}': {e}")
                finally:
                    workbook.close()  # Close the workbook explicitly

            else:
                print(f"Sheet '{sheet}' only exists in {file1}")

    # Check for sheets only in file2
    for sheet in visible_sheets2:
        if sheet not in visible_sheets1:
            print(f"Sheet '{sheet}' only exists in {file2}")

# Example usage
if __name__ == "__main__":
    file1 = r"C:\Users\nrsh\old_pc\Desktop\Study\spreadsheet_compare\06Geometry_J109_JE7-Final_HIGS_NRSH_V4.xlsx"
    file2 = r"C:\Users\nrsh\old_pc\Desktop\Study\spreadsheet_compare\07Geometry_J109_JE7-Final_HIGS_NRSH_V5.xlsx"

    compare_excel_files(file1, file2)
